# Cách tinh chỉnh mô hình LLM với bộ điều hợp LoRA sử dụng Hugging Face TRL

Notebook này trình bày cách tinh chỉnh hiệu quả các mô hình ngôn ngữ lớn sử dụng bộ điều hợp LoRA (Thích nghi hạng thấp). LoRA là một kỹ thuật tinh chỉnh hiệu quả về tham số với các đặc điểm:
- Đóng băng trọng số mô hình đã huấn luyện trước
- Thêm các ma trận phân rã hạng nhỏ có thể huấn luyện vào các lớp attention
- Thường giảm khoảng 90% số tham số cần huấn luyện
- Duy trì hiệu suất mô hình trong khi vẫn tiết kiệm bộ nhớ

Chúng ta sẽ tìm hiểu:
1. Thiết lập môi trường phát triển và cấu hình LoRA
2. Tạo và chuẩn bị tập dữ liệu để huấn luyện bộ điều hợp
3. Tinh chỉnh sử dụng `trl` và `SFTTrainer` với bộ điều hợp LoRA
4. Kiểm thử mô hình và kết hợp bộ điều hợp (tùy chọn)

## 1. Thiết lập môi trường 

Bước đầu tiên là cài đặt Thư viện Hugging Face và Pyroch, bao gồm trl, transformers và datasets. Nếu bạn chưa nghe nói về trl, đừng lo lắng. Đây là một thư viện mới trên đầu transformers và datasets, giúp tinh chỉnh các LLM mở dễ dàng hơn.


In [ ]:
# Cài đặt các thư viện cần thiết
# !pip install transformers datasets trl huggingface_hub

# Đăng nhập vào Hugging Face

from huggingface_hub import login

login()

# Để tiện lợi, hãy tạo một biến môi trường lưu token của bạn như là HF_TOKEN

## 2. Load the dataset

In [13]:
# Tải bộ dữ liệu từ Hugging Face
from datasets import load_dataset

# TODO: hãy thay đổi path và name cho phù hợp với bộ dữ liệu bạn muốn tải
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. Tinh chỉnh LLM sử dụng `trl` và `SFTTrainer` với LoRA

[SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) từ `trl` cung cấp tích hợp với bộ điều hợp LoRA thông qua thư viện [PEFT](https://huggingface.co/docs/peft/en/index). Những lợi thế chính của cách thiết lập này bao gồm:

1. **Hiệu quả bộ nhớ**: 
   - Chỉ các tham số bộ điều hợp được lưu trong bộ nhớ GPU
   - Trọng số mô hình cơ sở vẫn được đóng băng và có thể được tải với độ chính xác thấp hơn
   - Cho phép tinh chỉnh các mô hình lớn trên GPU thông dụng

2. **Tính năng huấn luyện**:
   - Tích hợp sẵn PEFT/LoRA với thiết lập tối thiểu
   - Hỗ trợ QLoRA (LoRA lượng tử hóa) để hiệu quả bộ nhớ tốt hơn nữa

3. **Quản lý bộ điều hợp**:
   - Lưu trọng số bộ điều hợp trong các điểm kiểm tra
   - Các tính năng để kết hợp bộ điều hợp trở lại mô hình cơ sở

Chúng ta sẽ sử dụng LoRA trong ví dụ của mình, kết hợp LoRA với lượng tử hóa 4-bit để giảm thêm mức sử dụng bộ nhớ mà không ảnh hưởng đến hiệu suất. Thiết lập chỉ yêu cầu một vài bước cấu hình:
1. Xác định cấu hình LoRA (hạng, alpha, dropout)
2. Tạo SFTTrainer với cấu hình PEFT
3. Huấn luyện và lưu trọng số bộ điều hợp


In [ ]:
# Import các thư viện cần thiết
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Tải mô hình và tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Tinh chỉnh định dạng
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Đặt tên cho mô hình tinh chỉnh
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

`SFTTrainer` hỗ trợ tích hợp sẵn với `peft`, điều này giúp việc tinh chỉnh hiệu quả các mô hình LLM trở nên cực kỳ dễ dàng, ví dụ như sử dụng LoRA. Chúng ta chỉ cần tạo `LoraConfig` và cung cấp nó cho trình huấn luyện.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập: Xác định tham số LoRA cho tinh chỉnh</h2>
    <p>Lấy một tập dữ liệu từ Hugging Face hub và tinh chỉnh một mô hình trên đó.</p>
    <p><b>Các mức độ khó</b></p>
    <p>🐢 Sử dụng các tham số chung cho một lần tinh chỉnh bất kỳ</p>
    <p>🐕 Điều chỉnh các tham số và đánh giá trong weights & biases</p>
    <p>🦁 Điều chỉnh các tham số và hiển thị sự thay đổi trong kết quả suy luận</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO: Tinh chỉnh các tham số LoRA
# r: hạng của ma trận LoRA (thường nằm trong khoảng 4-32) , nhỏ hơn tức là sẽ nén nhiều hơn
rank_dimension = 6
# lora_alpha: hệ số tỷ lệ cho các lớp LoRA (cao = sự thích nghi mạnh mẽ hơn)
lora_alpha = 8
# lora_dropout: tỉ lệ dropout cho các lớp LoRA (giúp mô hình tránh tình trạng overfitting)

lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # hạng của ma trận LoRA
    lora_alpha=lora_alpha,  # hệ số tỷ lệ cho các lớp LoRA
    lora_dropout=lora_dropout,  # tỉ lệ dropout cho các lớp LoRA
    bias="none",  # Loại bias cho LoRA, những bias sẽ được cập nhật trong quá trình huấn luyện
    target_modules="all-linear",  # Những module mà LoRA sẽ được áp dụng
    task_type="CAUSAL_LM",  # Loại bài toán mà mô hình đang giải quyết
)

Trước khi chúng ta có thể bắt đầu huấn luyện, chúng ta cần xác định các siêu tham số (`TrainingArguments`) mà chúng ta muốn sử dụng.

In [ ]:
# Những cài đặt cho quá trình tinh chỉnh
# Những siêu tham số dựa vào gợi ý từ bài báo QLoRA
args = SFTConfig(
    # Cài đặt đầu ra
    output_dir=finetune_name,  # Đường dẫn lưu  mô hình tinh chỉnh
    # Thời gian huấn luyện
    num_train_epochs=1,  # Số epoch
    # Cài đặt batch_size
    per_device_train_batch_size=2,  # Batch size trên mỗi GPU
    gradient_accumulation_steps=2,  # Accumulate gradients cho batch_size lớn hơn
    # Tối ưu bộ nhớ
    gradient_checkpointing=True,  # Sử dụng gradient checkpointing để giảm bộ nhớ nhưng tăng thời gian huấn luyện
    # Tinh chỉnh tối ưu hóa
    optim="adamw_torch_fused",  # Sử dụng AdamW
    learning_rate=2e-4,  # Hệ số learning_rate (QLoRA paper)
    max_grad_norm=0.3,  # Giới hạn gradient lớn nhất
    # Cài đặt learning rate
    warmup_ratio=0.03,  # Phần trăm số bước học tăng dần
    lr_scheduler_type="constant",  # Giữ nguyên learning rate
    # Logging và lưu trữ
    logging_steps=10,  # In ra thông tin sau mỗi bước
    save_strategy="epoch",  # Lưu mô hình sau mỗi epoch
    # Precision settings
    bf16=True,  # Sử dụng bfloat16
    # Những cài đặt khác
    push_to_hub=False,  # Không tải lên HuggingFace Hub
    report_to="none",  # Không gửi kết quả lên các nền tảng khác
)

Bây giờ chúng ta có mọi khối cần thiết để tạo `SFTTrainer` bắt đầu đào tạo mô hình của chúng ta.

In [ ]:
max_seq_length = 1512  # độ dài chuỗi tối đa cho mô hình và đóng gói bộ dữ liệu

# Tạo SFTTrainer với cấu hình LoRA
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # Cấu hình LoRA
    max_seq_length=max_seq_length,  # Độ dài chuỗi tối đa
    tokenizer=tokenizer,
    packing=True,  # Bật đóng gói đầu vào để tăng hiệu quả
    dataset_kwargs={
        "add_special_tokens": False,  # Các token đặc biệt được xử lý bởi template
        "append_concat_token": False,  # Không cần thêm ký tự phân tách
    },
)

Bắt đầu huấn luyện mô hình của chúng ta bằng cách gọi phương thức `train()` trên đối tượng `Trainer`. Điều này sẽ bắt đầu vòng lặp huấn luyện và huấn luyện mô hình của chúng ta trong 3 epoch. Vì chúng ta đang sử dụng phương pháp PEFT, chúng ta sẽ chỉ lưu các trọng số mô hình đã được điều chỉnh và không lưu toàn bộ mô hình.

In [ ]:
# bắt đầu huấn luyện
trainer.train()

# lưu mô hình
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

Việc huấn luyện với Flash Attention trong 3 epoch với tập dữ liệu gồm 15k mẫu mất 4:14:36 trên `g5.2xlarge`. Chi phí của phiên bản này là `1.21$/h`, tổng chi phí chỉ khoảng ~`5.3$`.


### Hợp nhất adapter LoRA vào mô hình gốc

Khi sử dụng LoRA, chúng ta chỉ huấn luyện các trọng số của adapter trong khi giữ nguyên mô hình gốc. Trong quá trình huấn luyện, chúng ta chỉ lưu các trọng số nhẹ của adapter (~2-10MB) thay vì một bản sao đầy đủ của mô hình. Tuy nhiên, để triển khai, bạn có thể muốn hợp nhất các adapter trở lại mô hình gốc vì:

1. **Triển khai đơn giản**: Một tệp mô hình thay vì mô hình gốc + bộ adapter
2. **Tốc độ suy luận**: Không cần tính toán adapter
3. **Tương thích với framework**: Tương thích tốt hơn với các framework


In [ ]:
from peft import AutoPeftModelForCausalLM


# Tải PEFT vào CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Kết hợp các trọng số trong Adapter và LoRA sau đó lưu mô hình
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. Kiểm thử mô hình và chạy thử

Sau khi huấn luyện xong, chúng ta muốn kiểm thử mô hình của mình. Chúng ta sẽ tải các mẫu khác nhau từ bộ dữ liệu gốc và đánh giá mô hình trên các mẫu đó, sử dụng một vòng lặp đơn giản và độ chính xác làm thước đo.


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập bổ sung: tải LoRA Adapter</h2>
    <p>Sử dụng những gì bạn học được từ ghi chú để tải LoRA Adapter đã được đào tạo của bạn để chạy</p> 
</div>

In [30]:
# dọn dẹp bộ nhớ
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

Hãy thử một số ví dụ để xem mô hình hoạt động

In [34]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)